In [1]:
from influxdb import InfluxDBClient
import utility as util
from datetime import datetime, timedelta
import pandas as pd
from pathlib import Path
import time
from decouple import config
from datetime import datetime, timedelta
import arrow
import slack

In [2]:
# Access the .env file Variables
from dotenv import load_dotenv
%load_ext dotenv
%dotenv

In [3]:
client = slack.WebClient(token = config('SLACK_TOKEN'))

In [3]:
class Influx():
    def __init__(self):
        self.user = config('INFLUXDB_USERNAME')
        self.password = config('INFLUXDB_PASSWORD')
        self.host = config('INFLUX_HOST')
        self.dbname = 'gateway-generic'  # 'linklab-users'
        self.port = 443
        self.ssl = True
        self.client = self.get_client()

    def get_client(self, dbname='gateway-generic'):
        client = InfluxDBClient(host=self.host, port=self.port, username=self.user,
                                password=self.password, database=self.dbname, ssl=self.ssl)
#         print('Retrieving client for: %s' % dbname)
        self.client = client
        return client

    def get_result_set(self, fieldname, add_param):  # show in new readme
        q_str = 'SELECT * FROM "%s" WHERE %s' % (fieldname, add_param)
        client = self.client
        result_set = client.query(q_str)
        return result_set

    def get_device_query_adds(self, device_id_list):
        q_append = ''
        count = 0
        for device_id in device_id_list:
            if count == 0:
                q_append += 'and ("device_id"=\'%s\'' % device_id
            else:
                q_append += 'or "device_id"=\'%s\'' % device_id
            count += 1
        q_append += ')'
        return q_append 

    # python datetime objects
    def get_time_query_from_now(self):
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Current Time:", current_time)
        # tq = "time >= '%s' and time < '%s'" % (start_time, end_time)
        tq = "time > now() -1m "
        return tq
    
def get_awair_data(fieldname, device_id):
    x = Influx()
    a = x.get_time_query_from_now()
    print('Retrieving device: %s, field: %s' % (device_id,fieldname))
    if device_id is not None:
        a += x.get_device_query_adds([device_id])
    
    r = x.get_result_set(fieldname, a)
    
    if util.get_longform_df(r) is None:
        return None
    else: 
        return util.get_longform_df(r).tail(1)

In [4]:
filepath = Path('linklab_awair_with_px.csv')
df = pd.read_csv(filepath)

In [10]:
# insecure-requestwarnings-be-disabled-in-the-requests-module
from requests.packages import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [11]:
device_id = '70886b123962'
fieldname = 'co2_ppm'
get_awair_data(fieldname, device_id )
# print(qs)
# print(f"Awair {device_id} '{fieldname}' value:{qs['value'].to_string(index=False)}%")


Current Time: 15:11:02
Retrieving device: 70886b123962, field: co2_ppm


,time,0,1,2,3,awair_device_uuid,awair_mac_address,description,details,device_class,device_id,facilities_tag,gateway_id,location_general,location_specific,received_time,receiver,value
5,2023-03-20 15:10:59-04:00,None,None,None,None,awair-omni_16469,None,Table 3,None,undefined,70886b123962,None,c0:98:e5:c0:00:38,UVA,211 Olsson,None,http-awairlocal-publish,514


In [5]:
res = {}
for idx , row in df.iterrows():
    device_id = row['device_id']
    fieldname = 'battery_%'
    qs = get_awair_data(fieldname, device_id ) 
    if qs is None:
        res[row['device_id']]= { 'location_specific':row['location_specific'], 'value':None, 'coordinate':row['pixel_coordinate']}
    else:
        res[row['device_id']]= { 'location_specific':row['location_specific'], 'value':int(qs['value'].to_string(index=False))}
#     print(j['device_id'], j["location_specific"], j['description'])
# client.chat_postMessage(channel='#awair-reminder', text=res)

Current Time: 13:52:52
Retrieving device: 70886b125d88, field: battery_%
Current Time: 13:52:52
Retrieving device: 70886b122593, field: battery_%
Current Time: 13:52:52
Retrieving device: 70886b123c0b, field: battery_%
Current Time: 13:52:52
Retrieving device: 70886b12335b, field: battery_%
Current Time: 13:52:52
Retrieving device: 70886b126a0e, field: battery_%
Current Time: 13:52:52
Retrieving device: 70886b1234a0, field: battery_%


/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/pytho

Current Time: 13:52:52
Retrieving device: 70886b123735, field: battery_%
Current Time: 13:52:52
Retrieving device: 70886b1238b5, field: battery_%
Current Time: 13:52:52
Retrieving device: 70886b12395b, field: battery_%
Current Time: 13:52:52
Retrieving device: 70886b123962, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b123992, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b123b81, field: battery_%


/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/pytho

Current Time: 13:52:53
Retrieving device: 70886b123b83, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b1275bc, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b127639, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b127730, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b12778c, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b1268c6, field: battery_%


...

/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/pytho

Current Time: 13:52:53
Retrieving device: 70886b1232f1, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b123368, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b1235d7, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b126223, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b1277ca, field: battery_%
Current Time: 13:52:53
Retrieving device: 70886b123059, field: battery_%


/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/pytho

Current Time: 13:52:53
Retrieving device: 70886b1230f2, field: battery_%
Current Time: 13:52:54
Retrieving device: 70886b1235d4, field: battery_%
Current Time: 13:52:54
Retrieving device: 70886b12762f, field: battery_%
Current Time: 13:52:54
Retrieving device: 70886b123c41, field: battery_%
Current Time: 13:52:54
Retrieving device: 70886b125f1e, field: battery_%


/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/pytho

In [6]:
non_connection_sensors = {}
for i in res:
    if res[i]['value'] is None:
#         print(i)
        non_connection_sensors[res[i]['location_specific']] = i

In [7]:
print(res)

{'70886b125d88': {'location_specific': '203 Olsson', 'value': 100}, '70886b122593': {'location_specific': '204 Olsson', 'value': 96}, '70886b123c0b': {'location_specific': 'C210 Olsson', 'value': None, 'coordinate': '575,282'}, '70886b12335b': {'location_specific': 'C211 Olsson', 'value': 100}, '70886b126a0e': {'location_specific': '211 Olsson', 'value': 100}, '70886b1234a0': {'location_specific': '211 Olsson', 'value': 100}, '70886b123735': {'location_specific': '211 Olsson', 'value': 95}, '70886b1238b5': {'location_specific': '211 Olsson', 'value': 100}, '70886b12395b': {'location_specific': '211 Olsson', 'value': 97}, '70886b123962': {'location_specific': '211 Olsson', 'value': 99}, '70886b123992': {'location_specific': '211 Olsson', 'value': 97}, '70886b123b81': {'location_specific': '211 Olsson', 'value': 99}, '70886b123b83': {'location_specific': '211 Olsson', 'value': None, 'coordinate': '673,175'}, '70886b1275bc': {'location_specific': '211 Olsson', 'value': 100}, '70886b127639

In [8]:
print(non_connection_sensors)

{'C210 Olsson': '70886b123c0b', '211 Olsson': '70886b123b83', '221 Olsson': '70886b12349c', '251 Olsson': '70886b123368', '269 Olsson': '70886b126223'}


In [21]:
client.chat_postMessage(channel='#awair-reminder', 
                        text=f"Awair network connectivity | Link Lab\nAwair Omni: 41\nStatus:\n---------\nno connectivity:{non_connection_sensors}")

In [22]:
import pandas as pd
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image # use the python image library
from collections import defaultdict

In [63]:
import matplotlib.image as mpimg
# img = mpimg.imread('floorplan_resolution.png')
img = Image.open("floorplan_resolution.png")

plt.figure(figsize = (26,13))
plt.xticks(range(0,1110,50))
plt.yticks(range(0,564,50))

non_connection_sensors = {}
points = []
for i in res:
    if res[i]['value'] is None:
#         print(i)
        non_connection_sensors[res[i]['location_specific']] = i
        points.append(tuple(eval(res[i]['coordinate'])))

# Awair-Onmi: to draw a point on co-ordinate (200,300)


x = list(map(lambda x: x[0], points))
y = list(map(lambda x: x[1], points))
plt.scatter(x, y, color = 'red', s=200, label = f"Awair-Onmi: {len(points)}")


# plt.grid(True)

# for xy in zip(x, y):
#     plt.annotate('(%.0f, %.0f)' % xy, xy=xy, color='green', fontsize=20)

#     plt.annotate(f'({x},{y}) {des[0]}', xy=xy, color='red', fontsize=20)

plt.imshow(img)
plt.xlim([0, 1110])
plt.ylim([564, 0])

plt.legend(loc = 'upper left')
plt.title('linklab pixel coordinate')
plt.xlabel('Pixel x')
plt.ylabel('Pixel Y')

# plt.show()
plt.savefig('my_plot.png')
img = open("my_plot.png", 'rb').read()

client.chat_postMessage(channel='#awair-reminder', 
                        text=f"Awair network connectivity | Link Lab\nAwair Omni: 41\nStatus:\n---------\nno connectivity:{non_connection_sensors}")


# Send the image
client.files_upload(
        channels = "#awair-reminder",
        initial_comment = "That's one small step for man, one giant leap for mankind.",
        filename = "Apollo 11",
        content = img)

plt.clf()


<Figure size 1872x936 with 0 Axes>

In [65]:
print (type(img))

<class 'bytes'>


In [102]:
filepath = Path('1st_awair_with_px.csv')
df = pd.read_csv(filepath)

device_id = '70886b12624f'
fieldname = 'battery_%'
get_awair_data(fieldname, device_id )



Retrieving client for: gateway-generic
current Time: 18:30:57


/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,time,awair_device_uuid,description,details,device_class,device_id,gateway_id,location_general,location_specific,receiver,value
5,2023-03-13 18:30:47-04:00,awair-omni_26126,South Wall,None,undefined,70886b12624f,c0:98:e5:c0:00:38,UVA,112 Olsson,http-awairlocal-publish,100


In [113]:
res = {}
for idx , row in df.iterrows():
    device_id = row['device_id']
    fields = ['battery_%']
    for fieldname in fields:
        qs = get_awair_data(fieldname, device_id ) 
        if qs is None:
            res[row['device_id']]= { 'location_specific':row['location_specific'], 'value':None}
        else:
            res[row['device_id']]= { 'location_specific':row['location_specific'], 'value':int(qs['value'].to_string(index=False))}
#     print(j['device_id'], j["location_specific"], j['description'])
# client.chat_postMessage(channel='#awair-reminder', text=res)

Retrieving client for: gateway-generic
current Time: 14:49:56
Retrieving client for: gateway-generic
current Time: 14:49:57


/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Retrieving client for: gateway-generic
current Time: 14:49:57
Retrieving client for: gateway-generic
current Time: 14:49:57


/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Retrieving client for: gateway-generic
current Time: 14:49:57
Retrieving client for: gateway-generic
current Time: 14:49:57


/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Retrieving client for: gateway-generic
current Time: 14:49:57
Retrieving client for: gateway-generic
current Time: 14:49:57


/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/apple/opt/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [114]:
print(res)

{'70886b125beb': {'location_specific': '111H Olsson', 'value': 98}, '70886b125e64': {'location_specific': '105 Olsson', 'value': 98}, '70886b125c6f': {'location_specific': '114 Olsson', 'value': 95}, '70886b125db1': {'location_specific': 'C111 Olsson', 'value': 99}, '70886b12624f': {'location_specific': '112 Olsson', 'value': 100}, '70886b126a0d': {'location_specific': '111F', 'value': 100}, '70886b125d72': {'location_specific': '104 Olsson', 'value': 100}, '70886b127d5c': {'location_specific': 'C113 Olsson', 'value': 94}}


In [104]:
non_connection_sensors = {}
for i in res:
    if res[i]['value'] is None:
#         print(i)
        non_connection_sensors[res[i]['location_specific']] = i

print(non_connection_sensors)

{}


In [112]:
client.chat_postMessage(channel='#awair-reminder', 
                        text=f"Awair network connectivity | Olsson 1st\nAwair Omni: 8\nStatus:\n---------\nno connectivity:None")